In [1]:
import get_nirspec_mos_info

# NIRSpec/MOS GO cycle 1-3

In [4]:
from get_nirspec_mos_info import get_nirspec_mos_basic_info_GO


urls = {
    "Cycle 1": "https://www.stsci.edu/jwst/science-execution/approved-programs/general-observers/cycle-1-go",
    "Cycle 2": "https://www.stsci.edu/jwst/science-execution/approved-programs/general-observers/cycle-2-go",
    "Cycle 3": "https://www.stsci.edu/jwst/science-execution/approved-programs/general-observers/cycle-3-go"
} # URLs for JWST GO Cycle 1, Cycle 2, and Cycle 3

get_nirspec_mos_basic_info_GO(urls)




Extracting data for Cycle 1...
Extracting data for Cycle 2...


KeyboardInterrupt: 

# Extract 'Status' of each proposal

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Function to get the observation status for a given Proposal ID
def get_observation_status(proposal_id, retries=3):
    url = f"https://www.stsci.edu/cgi-bin/get-visit-status?id={proposal_id}&markupFormat=html&observatory=JWST&pi=1"
    session = requests.Session()
    attempt = 0
    while attempt < retries:
        try:
            response = session.get(url, timeout=30)
            response.raise_for_status()  # Raise an error for bad status codes
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all tables with observation status
            tables = soup.find_all("table")
            if not tables:
                print(f"No tables found for Proposal ID {proposal_id}")
                return [], []  # Return two empty lists if no tables are found
            
            all_status_data = []
            all_headers = []  # Initialize as a list to maintain order
            
            for table in tables:
                # Get all rows in the table
                rows = table.find_all('tr')
                
                # Parse the table headers (first row)
                headers = [header.get_text(strip=True) for header in rows[0].find_all('td')]
                if headers:  # Ensure headers are not empty
                    all_headers.extend([header for header in headers if header not in all_headers])  # Add only new headers
                    print(f"Headers: {headers}")  # Debug: Output the headers
                
                    # Locate the "Template" column index
                    template_index = headers.index("Template") if "Template" in headers else -1
                    
                    # Parse the rows in the table (skip header)
                    rows = rows[1:]  # Skip the header row
                    print(f"Number of rows found: {len(rows)}")  # Debug: Output the number of rows found
                    
                    for row in rows:
                        columns = row.find_all('td')
                        
                        if len(columns) != len(headers):
                            continue  # Skip rows that don't match the header length
                        
                        row_data = [col.get_text(strip=True) for col in columns]
                        print(f"Row data: {row_data}")  # Debug: Output the data in this row
                        
                        # Check if the "Template" matches "NIRSpec MultiObject Spectroscopy"
                        if template_index != -1 and "NIRSpec MultiObject Spectroscopy" in row_data[template_index]:
                            row_dict = dict(zip(headers, row_data))  # Map header to row data
                            all_status_data.append(row_dict)
            
            return all_status_data, all_headers
        
        except (requests.ConnectionError, requests.Timeout) as e:
            print(f"Connection error for Proposal ID {proposal_id}: {e}. Retrying...")
            attempt += 1
            time.sleep(5)  # Wait before retrying
    
    print(f"Failed to fetch data for Proposal ID {proposal_id} after {retries} retries.")
    return [], []  # Return two empty lists in case of failure

# Load the existing CSV file
df = pd.read_csv('NIRSpec_MOS_pps_GO.csv')

# Prepare a DataFrame to hold the expanded data with status details
all_status_data = []
final_headers = list(df.columns)  # Start with the headers from the original dataframe

# Iterate through each proposal and fetch its observation status
for index, row in df.iterrows():
    proposal_id = row['ID']
    print(f"Fetching status for Proposal ID: {proposal_id}")
    
    status_data, headers = get_observation_status(proposal_id)
    
    if status_data:
        # Update the final headers with any new ones found
        for header in headers:
            if header not in final_headers:
                final_headers.append(header)
        
        for entry in status_data:
            # Create a dictionary for this row to map headers to their respective data
            new_row_dict = dict(zip(df.columns, row.tolist()))  # Convert row to dict
            new_row_dict.update(entry)  # Add/Update with status entry data
            
            # Append row in the correct header order
            new_row = [new_row_dict.get(header, '') for header in final_headers]
            all_status_data.append(new_row)
    
    # Add a delay to avoid overloading the server
    time.sleep(3)  # Increase delay between requests

# Create the final DataFrame and write to CSV
if all_status_data:
    df_status = pd.DataFrame(all_status_data, columns=final_headers)
    df_status.to_csv('NIRSpec_MOS_pps_GO_with_detailed_status.csv', index=False)
    print("Updated proposal data with detailed observation status has been written to NIRSpec_MOS_pps_GO_with_detailed_status.csv")
else:
    print("No NIRSpec MultiObject Spectroscopy data found.")


Fetching status for Proposal ID: 1433
Headers: ['Observation', 'Visit', 'Status', 'Targets', 'Template', 'Hours', 'Start UT', 'End UT', 'Repeat']
Number of rows found: 5
Row data: ['10', '1', 'Archived', 'MACS0647+7015', 'NIRCam Imaging', '3.31', 'Sep 23, 2022 11:42:17', 'Sep 23, 2022 15:05:56', '']
Row data: ['21', '1', 'Archived', 'MACS0647-MSA-TARGETS', 'NIRSpec MultiObject Spectroscopy', '3.82', 'Jan 8, 2023 06:23:57', 'Jan 8, 2023 09:17:00', '']
Row data: ['20', '1', 'Archived', 'MACS0647+7015', 'NIRCam Imaging', '1.12', 'Jan 8, 2023 16:57:45', 'Jan 8, 2023 18:27:01', '']
Row data: ['23', '1', 'Archived', 'MACS0647-MSA-OBS23', 'NIRSpec MultiObject Spectroscopy', '3.83', 'Feb 20, 2023 17:57:32', 'Feb 20, 2023 21:14:48', 'Repeat of observation 22 visit 1 in this program by WOPR88662']
Row data: ['22', '1', 'FailedArchived', 'MACS0647-MSA-TARGETS', 'NIRSpec MultiObject Spectroscopy', '3.11', 'Jan 8, 2023 04:59:27', 'Jan 8, 2023 06:23:53', 'Rescheduled\nby WOPR88662as observation 23 v

# NIRSpec/MOS GTO

In [4]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Function to extract NIRSpec/MOS data from the GTO page
def extract_nirspec_mos_gto_data(url):
    # Set up Selenium WebDriver
    driver = webdriver.Chrome()
    driver.get(url)

    # Wait for the page to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, 'table'))
    )

    # Get the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()

    # Parse the content to extract proposals from all tables
    tables = soup.find_all('table')  # Find all tables
    data = []
    headers = None

    for table in tables:
        rows = table.find_all('tr')
        if not rows:
            continue  # Skip empty tables

        # Assuming first row is headers
        current_headers = [header.get_text(strip=True) for header in rows[0].find_all('th')]
        
        if headers is None:  # Set headers only once
            headers = current_headers
        
        for row in rows[1:]:
            columns = row.find_all('td')
            if len(columns) != len(headers):
                continue  # Skip rows that do not match the header length

            # Check if "Instrument/Mode" column exists
            if "Instrument/Mode" in headers:
                instrument_mode = columns[headers.index("Instrument/Mode")].get_text(strip=True)
            elif "Instrument/ Mode" in headers:
                instrument_mode = columns[headers.index("Instrument/ Mode")].get_text(strip=True)
            else:
                continue  # Skip if there's no Instrument/Mode column

            # Only add rows with "NIRSpec/MOS"
            if "NIRSpec/MOS" in instrument_mode:
                row_data = [col.get_text(strip=True) for col in columns]
                data.append(row_data)

    return data, headers

# URL for the GTO page
url = "https://www.stsci.edu/jwst/science-execution/approved-programs/guaranteed-time-observations"

# Extract NIRSpec/MOS data from the GTO page
nirspec_data, nirspec_headers = extract_nirspec_mos_gto_data(url)

# Save the extracted data to a CSV file
if nirspec_data:
    df = pd.DataFrame(nirspec_data, columns=nirspec_headers)
    df.to_csv('NIRSpec_MOS_pps_GTO.csv', index=False)
    print("NIRSpec/MOS GTO proposal data has been written to NIRSpec_MOS_pps_GTO.csv")
else:
    print("No NIRSpec/MOS data found.")


NIRSpec/MOS GTO proposal data has been written to NIRSpec_MOS_pps_GTO.csv


In [5]:
# extract 'status' data from the GTO page
# Load the existing CSV file
df = pd.read_csv('NIRSpec_MOS_pps_GTO.csv')

# Prepare a DataFrame to hold the expanded data with status details
all_status_data = []
final_headers = list(df.columns)  # Start with the headers from the original dataframe

# Iterate through each proposal and fetch its observation status
for index, row in df.iterrows():
    proposal_id = row['ID']
    print(f"Fetching status for Proposal ID: {proposal_id}")
    
    status_data, headers = get_observation_status(proposal_id)
    
    if status_data:
        # Update the final headers with any new ones found
        for header in headers:
            if header not in final_headers:
                final_headers.append(header)
        
        for entry in status_data:
            # Create a dictionary for this row to map headers to their respective data
            new_row_dict = dict(zip(df.columns, row.tolist()))  # Convert row to dict
            new_row_dict.update(entry)  # Add/Update with status entry data
            
            # Append row in the correct header order
            new_row = [new_row_dict.get(header, '') for header in final_headers]
            all_status_data.append(new_row)
    
    # Add a delay to avoid overloading the server
    time.sleep(3)  # Increase delay between requests

# Create the final DataFrame and write to CSV
if all_status_data:
    df_status = pd.DataFrame(all_status_data, columns=final_headers)
    df_status.to_csv('NIRSpec_MOS_pps_GTO_with_detailed_status.csv', index=False)
    print("Updated proposal data with detailed observation status has been written to NIRSpec_MOS_pps_GTO_with_detailed_status.csv")
else:
    print("No NIRSpec MultiObject Spectroscopy data found.")

Fetching status for Proposal ID: 4527
Headers: ['Observation', 'Visit', 'Status', 'Targets', 'Template', 'Hours', 'Plan Windows']
Number of rows found: 1
Row data: ['2', '1', 'Implementation', 'macs1149ncf-photutils-v2p0p2', 'NIRSpec MultiObject Spectroscopy', '5.96', 'May 15, 2025 - Jun 13, 2025 (2025.135 - 2025.164)']
Fetching status for Proposal ID: 4552
Headers: ['Observation', 'Visit', 'Status', 'Targets', 'Template', 'Hours', 'Plan Windows']
Number of rows found: 2
Row data: ['1', '1', 'Implementation', 'v16', 'NIRSpec MultiObject Spectroscopy', '6.38', 'May 11, 2025 - Jun 11, 2025 (2025.131 - 2025.162)']
Row data: ['2', '1', 'Implementation', 'v16', 'NIRSpec MultiObject Spectroscopy', '7.07', 'May 11, 2025 - Jun 11, 2025 (2025.131 - 2025.162)']
Fetching status for Proposal ID: 2758
Headers: ['Observation', 'Visit', 'Status', 'Targets', 'Template', 'Hours', 'Start UT', 'End UT']
Number of rows found: 1
Row data: ['21', '1', 'Archived', 'TM_M1149PAR_v10_fin', 'NIRSpec MultiObject 

# check

In [ ]:
# 对比NIRSpec_MOS_pps_GTO.csv和NIRSpec_MOS_pps_GTO_with_detailed_status.csv中“ID”列，找出第一个文件有而第二个文件没有的id号
# Load the original GTO data
df_gto = pd.read_csv('NIRSpec_MOS_pps_GTO.csv')
df_gto_status = pd.read_csv('NIRSpec_MOS_pps_GTO_with_detailed_status.csv')


In [8]:
import pandas as pd
import os

def get_observation_status_2(proposal_id, retries=3):
    url = f"https://www.stsci.edu/cgi-bin/get-visit-status?id={proposal_id}&markupFormat=html&observatory=JWST&pi=1"
    session = requests.Session()
    attempt = 0
    while attempt < retries:
        try:
            response = session.get(url, timeout=60)
            response.raise_for_status()  # Raise an error for bad status codes
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all tables with observation status
            tables = soup.find_all("table")
            if not tables:
                print(f"No tables found for Proposal ID {proposal_id}")
                return [], []  # Return two empty lists if no tables are found
            
            all_status_data = []
            all_headers = []  # Initialize as a list to maintain order
            
            for table in tables:
                # Get all rows in the table
                rows = table.find_all('tr')
                
                # Parse the table headers (first row)
                headers = [header.get_text(strip=True) for header in rows[0].find_all('td')]
                if headers:  # Ensure headers are not empty
                    all_headers.extend([header for header in headers if header not in all_headers])  # Add only new headers
                    print(f"Headers: {headers}")  # Debug: Output the headers
                
                    # Locate the "Template" column index
                    template_index = headers.index("Template") if "Template" in headers else -1
                    
                    # Parse the rows in the table (skip header)
                    rows = rows[1:]  # Skip the header row
                    print(f"Number of rows found: {len(rows)}")  # Debug: Output the number of rows found
                    
                    for row in rows:
                        columns = row.find_all('td')
                        
                        if len(columns) != len(headers):
                            continue  # Skip rows that don't match the header length
                        
                        row_data = [col.get_text(strip=True) for col in columns]
                        print(f"Row data: {row_data}")  # Debug: Output the data in this row
                        
                        # Check if the "Template" matches "NIRSpec MultiObject Spectroscopy"
                        if template_index != -1 and "NIRSpec MultiObject Spectroscopy" in row_data[template_index]:
                            row_dict = dict(zip(headers, row_data))  # Map header to row data
                            all_status_data.append(row_dict)
            
            return all_status_data, all_headers
        
        except (requests.ConnectionError, requests.Timeout) as e:
            print(f"Connection error for Proposal ID {proposal_id}: {e}. Retrying...")
            attempt += 1
            time.sleep(5)  # Wait before retrying
    
    print(f"Failed to fetch data for Proposal ID {proposal_id} after {retries} retries.")
    return [], []  # Return two empty lists in case of failure



Headers: ['Observation', 'Visit', 'Status', 'Targets', 'Template', 'Hours', 'Start UT', 'End UT', 'Repeat']
Number of rows found: 20
Row data: ['1', '1', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.87', 'Dec 7, 2022 08:42:33', 'Dec 7, 2022 12:45:00', '']
Row data: ['1', '3', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.88', 'Dec 11, 2022 09:52:24', 'Dec 11, 2022 14:01:44', '']
Row data: ['1', '4', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.86', 'Dec 11, 2022 14:01:48', 'Dec 11, 2022 17:17:49', '']
Row data: ['1', '5', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '4.52', 'Dec 11, 2022 17:17:53', 'Dec 11, 2022 20:38:45', '']
Row data: ['1', '7', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.87', 'Dec 11, 2022 21:00:20', 'Dec 12, 2022 00:23:26', '']
Row data: ['1', '8', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.87', 'Dec 12, 2022 00:23:30', 'Dec 12, 2022 03:46:12', '']
Row data: ['1', '12', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.86', 'Dec 12, 2022 17:07:36', '

In [9]:

update_csv_with_single_proposal(
    proposal_id=2073, 
    original_csv_filename='NIRSpec_MOS_pps_GO.csv', 
    updated_csv_filename='NIRSpec_MOS_pps_GO_with_detailed_status.csv'
)


Headers: ['Observation', 'Visit', 'Status', 'Targets', 'Template', 'Hours', 'Start UT', 'End UT']
Number of rows found: 20
Row data: ['1', '1', 'Archived', 'DESJ0252-0503', 'NIRCam Imaging', '0.65', 'Aug 21, 2022 22:24:33', 'Aug 22, 2022 00:00:16']
Row data: ['1', '2', 'Archived', 'DESJ0252-0503', 'NIRCam Imaging', '0.65', 'Aug 22, 2022 00:00:19', 'Aug 22, 2022 00:31:29']
Row data: ['1', '3', 'Archived', 'DESJ0252-0503', 'NIRCam Imaging', '0.65', 'Aug 22, 2022 00:31:33', 'Aug 22, 2022 01:03:35']
Row data: ['1', '4', 'Archived', 'DESJ0252-0503', 'NIRCam Imaging', '0.70', 'Aug 22, 2022 01:03:38', 'Aug 22, 2022 01:34:48']
Row data: ['1', '5', 'Archived', 'DESJ0252-0503', 'NIRCam Imaging', '1.27', 'Aug 22, 2022 01:34:52', 'Aug 22, 2022 02:15:10']
Row data: ['1', '7', 'Archived', 'DESJ0252-0503', 'NIRCam Imaging', '0.65', 'Aug 22, 2022 02:46:28', 'Aug 22, 2022 03:18:31']
Row data: ['1', '8', 'Archived', 'DESJ0252-0503', 'NIRCam Imaging', '0.65', 'Aug 22, 2022 03:18:34', 'Aug 22, 2022 03:49:

In [10]:
import pandas as pd
import os
import time

# Function to fetch status data for a single proposal and update the CSV file
def update_csv_with_single_proposal(proposal_id, original_csv_filename, updated_csv_filename):
    # Load the existing updated CSV file
    if os.path.exists(updated_csv_filename):
        df_updated = pd.read_csv(updated_csv_filename)
        final_headers = list(df_updated.columns)
    else:
        print(f"CSV file {updated_csv_filename} not found.")
        return
    
    # Load the original CSV to get original proposal data
    df_original = pd.read_csv(original_csv_filename)
    original_row = df_original[df_original['ID'] == proposal_id]
    
    if original_row.empty:
        print(f"No data found in the original file for Proposal ID {proposal_id}.")
        return
    
    # Convert the original row to a dictionary (assuming only one match)
    original_row_dict = original_row.iloc[0].to_dict()

    # Fetch observation status for the given proposal_id
    status_data, headers = get_observation_status_2(proposal_id)
    
    if status_data:
        all_status_data = []
        for entry in status_data:
            # Create a new row by combining the original data with the status entry
            new_row_dict = original_row_dict.copy()  # Start with original data
            new_row_dict.update(dict(zip(headers, entry)))  # Add/Update with status entry data
            
            # Ensure the row is in the correct header order
            new_row = [new_row_dict.get(header, '') for header in final_headers]
            all_status_data.append(new_row)
        
        # Append new rows to the updated DataFrame
        df_new_entries = pd.DataFrame(all_status_data, columns=final_headers)
        df_updated = pd.concat([df_updated, df_new_entries], ignore_index=True)
        
        # Save the updated CSV
        df_updated.to_csv(updated_csv_filename, index=False)
        print(f"Proposal ID {proposal_id} has been added to {updated_csv_filename}")
    else:
        print(f"No NIRSpec MultiObject Spectroscopy data found for Proposal ID {proposal_id}.")

# Example usage
update_csv_with_single_proposal(
    proposal_id=1207, 
    original_csv_filename='NIRSpec_MOS_pps_GTO.csv', 
    updated_csv_filename='NIRSpec_MOS_pps_GTO_with_detailed_status.csv'
)


Headers: ['Observation', 'Visit', 'Status', 'Targets', 'Template', 'Hours', 'Start UT', 'End UT', 'Repeat']
Number of rows found: 20
Row data: ['1', '1', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.87', 'Dec 7, 2022 08:42:33', 'Dec 7, 2022 12:45:00', '']
Row data: ['1', '3', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.88', 'Dec 11, 2022 09:52:24', 'Dec 11, 2022 14:01:44', '']
Row data: ['1', '4', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.86', 'Dec 11, 2022 14:01:48', 'Dec 11, 2022 17:17:49', '']
Row data: ['1', '5', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '4.52', 'Dec 11, 2022 17:17:53', 'Dec 11, 2022 20:38:45', '']
Row data: ['1', '7', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.87', 'Dec 11, 2022 21:00:20', 'Dec 12, 2022 00:23:26', '']
Row data: ['1', '8', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.87', 'Dec 12, 2022 00:23:30', 'Dec 12, 2022 03:46:12', '']
Row data: ['1', '12', 'Archived', 'XDF-OFFCENTER', 'MIRI Imaging', '3.86', 'Dec 12, 2022 17:07:36', '